# Pose Notebook
In this notebook the Ufactory lite6 gripper is controlled through rotation angles but not really in a fashion way beacuse the reference frame is dynamic and not fixed, so it is really hard to give as input a certain position.
This aspect needs to be fixed as soon as possibile

**packages imports** 


In [34]:
#!/usr/bin/env python3
import time
import rclpy
from rclpy.logging import get_logger
#import moveit python library
from moveit.core.robot_state import RobotState
from moveit.planning import (MoveItPy,MultiPipelinePlanRequestParameters)

import numpy as np
from geometry_msgs.msg import PoseStamped, Pose    # set pose goal with PoseStamped messsage

from math import pi

from moveit_configs_utils import MoveItConfigsBuilder
from ament_index_python.packages import get_package_share_directory
import random

**plan and execute function**

- *Helper function to plan and execute a motion.*
 

In [2]:
def plan_and_execute(
    robot,
    planning_component,
    logger,
    single_plan_parameters=None,
    multi_plan_parameters=None,
    sleep_time=0.0,
):
    # plan to goal
    logger.info("Planning trajectory")
    if multi_plan_parameters is not None:
        plan_result = planning_component.plan(
            multi_plan_parameters=multi_plan_parameters
        )
    elif single_plan_parameters is not None:
        plan_result = planning_component.plan(
            single_plan_parameters=single_plan_parameters
        )
    else:
        plan_result = planning_component.plan()

    # execute the plan
    if plan_result:
        logger.info("Executing plan")
        robot_trajectory = plan_result.trajectory
        robot.execute(robot_trajectory, controllers=[])
    else:
        logger.error("Planning failed")

    time.sleep(sleep_time)


**convert euler angles to quaternions**

- Convert angles in radiants to a list of quaternions.

In [3]:
def get_quaternion_from_euler(roll, pitch, yaw):

  qx = np.sin(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) - np.cos(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
  qy = np.cos(roll/2) * np.sin(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.cos(pitch/2) * np.sin(yaw/2)
  qz = np.cos(roll/2) * np.cos(pitch/2) * np.sin(yaw/2) - np.sin(roll/2) * np.sin(pitch/2) * np.cos(yaw/2)
  qw = np.cos(roll/2) * np.cos(pitch/2) * np.cos(yaw/2) + np.sin(roll/2) * np.sin(pitch/2) * np.sin(yaw/2)
 
  return [qx, qy, qz, qw]

**set goal function**

- Set the goal pose in quaternions.

In [4]:
def set_goal_from_angles(roll, pitch, yaw): 
    # ATTENZIONE! " I would suggest using something like numpy-quaternion for making sure your input quaternion is valid and properly normalized, since it's very easy to run into precision issues"
    # Se i valori delle pose non sono ben normalizzati, il comando non diventa valido!
    # set start state to current state
    lite6_arm.set_start_state_to_current_state()

    quaternion = get_quaternion_from_euler (roll, pitch, yaw)

    pose_goal = Pose()    
    pose_goal.orientation.x = quaternion[0]
    pose_goal.orientation.y = quaternion[1]
    pose_goal.orientation.z = quaternion[2]
    pose_goal.orientation.w = quaternion[3]


**Configure MoveitPy**

- Configures MoveitPy oject with yamls files such as description, trajectory execution, semantic, kinematics, joint limits and planning config

In [5]:
moveit_config = (
        MoveItConfigsBuilder(robot_name="UF_ROBOT", package_name="lite6_enrico")
        .robot_description_semantic(file_path="config/UF_ROBOT.srdf")
        .trajectory_execution(file_path="config/moveit_controllers.yaml")
        .robot_description(file_path="config/UF_ROBOT.urdf.xacro")
        .robot_description_kinematics(file_path="config/kinematics.yaml")
        .joint_limits(file_path="config/joint_limits.yaml")
        .moveit_cpp(
            file_path=get_package_share_directory("lite6_moveit_demos")
            + "/config/moveit_cpp.yaml"
        )
        .to_moveit_configs()
    ).to_dict()

**initialise MoveitPY**

- Initializes MoveitPy node given the configuration builder set in the previous cell

In [6]:
rclpy.init() # initialise rclpy (only for logging purposes)
logger = get_logger("moveit_py.pose_goal")

# instantiate moveit_py instance and a planning component for the panda_arm
lite6 = MoveItPy(node_name="moveit_py", config_dict=moveit_config)
lite6_arm = lite6.get_planning_component("lite6_arm")

[INFO] [1716219583.786173341] [moveit_879846967.moveit.py.cpp_initializer]: Initialize rclcpp
[INFO] [1716219583.786233263] [moveit_879846967.moveit.py.cpp_initializer]: Initialize node parameters
[INFO] [1716219583.786239695] [moveit_879846967.moveit.py.cpp_initializer]: Initialize node and executor
[INFO] [1716219583.800635876] [moveit_879846967.moveit.py.cpp_initializer]: Spin separate thread
[INFO] [1716219583.804562298] [moveit_879846967.moveit.ros.rdf_loader]: Loaded robot model in 0.00378598 seconds
[INFO] [1716219583.804588857] [moveit_879846967.moveit.core.robot_model]: Loading robot model 'UF_ROBOT'...
[INFO] [1716219583.804594458] [moveit_879846967.moveit.core.robot_model]: No root/virtual joint specified in SRDF. Assuming fixed joint
[WARN] [1716219583.826554149] [moveit_879846967.moveit.core.robot_model]: Could not identify parent group for end-effector 'link_tcp'
[INFO] [1716219583.836322784] [moveit_879846967.moveit.kinematics.kdl_kinematics_plugin]: Joint weights for gr

**Go to pose definition**

- This function go_to_pose(roll, pitch, yaw) is used to move a robot arm to a specific pose defined by the roll, pitch, and yaw parameters. Here's a step-by-step explanation:

1. The function starts by setting plan to False. This variable is used later to check if the robot arm was able to find a valid pose.

2. It gets a planning scene monitor from the robot arm. This monitor is used to get the current state of the robot and to set the new pose.

3. It creates a RobotState instance using the current state of the robot. This state includes the current positions of the robot's joints.

4. It sets the start state of the robot arm to its current state.

5. It gets the current pose of the robot arm's end effector (the "link_tcp").

6. It prints the initial pose, the roll, pitch, yaw values, and the corresponding quaternion.

7. creates a new pose goal based on the current pose but with the orientation defined by the quaternion calculated from the roll, pitch, and yaw.

8. tries to set the robot state to the new pose using inverse kinematics (IK). If it can't find a valid pose that doesn't collide with anything, it logs an error message and returns from the function.

9.  a valid pose is found, it sets the goal state of the robot arm to the new pose and updates the robot state.

10. It gets the updated pose of the robot arm's end effector and prints it.

11. It logs a message indicating that the robot arm is moving to the goal pose.

12. It sets the joint positions of the robot arm back to their original positions and updates the robot state.

13. If a valid pose was found (i.e., plan is True), it calls a function plan_and_execute to move the robot arm to the new pose.

In [33]:
from geometry_msgs.msg import PoseStamped, Pose    # set pose goal with PoseStamped messsage
def go_to_pose(roll, pitch, yaw, movx, movy, movz):
        plan = False
        planning_scene_monitor = lite6.get_planning_scene_monitor()
        #with planning_scene_monitor.read_write() as scene:
        with planning_scene_monitor.read_only() as scene:

            # instantiate a RobotState instance using the current robot model
            robot_state = scene.current_state
            original_joint_positions = robot_state.get_joint_group_positions("lite6_arm")

            lite6_arm.set_start_state_to_current_state()

            check_init_pose = robot_state.get_pose("link_tcp")
            print("Initial_pose:", check_init_pose)
            print("roll is", roll,"pitch is", pitch,"yaw is", yaw)
            print("Quaternions are", get_quaternion_from_euler(roll, pitch, yaw))

            quaternion = get_quaternion_from_euler (roll, pitch, yaw)

            pose_goal = Pose()
            pose_goal.position.x = check_init_pose.position.x  + movx
            pose_goal.position.y = check_init_pose.position.y  + movy
            pose_goal.position.z = check_init_pose.position.z  + movz
            pose_goal.orientation.x = quaternion[0]
            pose_goal.orientation.y = quaternion[1]
            pose_goal.orientation.z = quaternion[2]
            pose_goal.orientation.w = quaternion[3]


            # Set the robot state and check collisions
            result = robot_state.set_from_ik("lite6_arm", pose_goal, "link_tcp", timeout=5.0)
            if not result:
                logger.error("IK solution was not found!")
                return
            else:
                logger.info("IK solution found!")
                plan = True
            
                lite6_arm.set_goal_state(robot_state=robot_state)   
            
                robot_state.update() # otherwise the kernel crashes

                check_updated_pose = robot_state.get_pose("link_tcp")

                print("New_pose:", check_updated_pose)

                logger.info("Go to goal")

                robot_state.set_joint_group_positions(
                "lite6_arm",
                original_joint_positions,
                    )
                robot_state.update()

        if plan == True:
            plan_and_execute(lite6, lite6_arm, logger,sleep_time=3.0) 

In [32]:
# Questa sotto è òa configurazzione di default, alla fine il primo parametro sarà sempre il tezo dovrà cambiare.
# l'apertura del gripper è sempre parallela al suo asse x
#go_to_pose(0,pi,0) questa è una configurazione di default con il gripper in basso 
#go_to_pose(0,pi,pi/2) # questa è una configurazione di default con il gripper in basso
go_to_pose(0,pi,random.uniform(-2*pi, 2*pi),random.uniform(-0.1, 0.1),random.uniform(-0.1, 0.1),random.uniform(-0.1, 0.1))


Initial_pose: geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.23668607690863605, y=0.019990374162415225, z=0.299972570101641), orientation=geometry_msgs.msg.Quaternion(x=0.258836389646137, y=0.9659211786657095, z=-9.260443476085657e-09, w=-9.607465908913046e-09))
roll is 0 pitch is 3.141592653589793 yaw is 0.5235987755982988
Quaternions are [-0.25881904510252074, 0.9659258262890683, 1.5848095757158825e-17, 5.914589856893349e-17]
New_pose: geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.23668609292074808, y=0.029990304438192818, z=0.3099721462315107), orientation=geometry_msgs.msg.Quaternion(x=-0.2588190445115125, y=0.9659258264474287, z=-8.393819420894436e-09, w=3.609961290463106e-10))


[INFO] [1716220416.641007566] [moveit_py.pose_goal]: IK solution found!
[INFO] [1716220416.647373692] [moveit_py.pose_goal]: Go to goal
[INFO] [1716220416.654249443] [moveit_py.pose_goal]: Planning trajectory
[WARN] [1716220416.654342297] [moveit_py]: Parameter 'plan_request_params.planning_time' not found in config use default value instead, check parameter type and namespace in YAML file
[INFO] [1716220416.654644012] [moveit_py]: Calling PlanningRequestAdapter 'ResolveConstraintFrames'
[INFO] [1716220416.654774988] [moveit_py]: Calling PlanningRequestAdapter 'ValidateWorkspaceBounds'
[WARN] [1716220416.654788824] [moveit_879846967.moveit.ros.validate_workspace_bounds]: It looks like the planning volume was not specified. Using default values.
[INFO] [1716220416.654849538] [moveit_py]: Calling PlanningRequestAdapter 'CheckStartStateBounds'
[INFO] [1716220416.654920280] [moveit_py]: Calling PlanningRequestAdapter 'CheckStartStateCollision'
[INFO] [1716220416.655058620] [moveit_87984696

in sostanza si applicano dei contraint ai gradi di libertà sulla terna dell'EE per cui la go_to_pose avrà sempre come primo parametro lo 0 mentre come secondo pi.

**Il prossimo passo è quello di implementare matrici di rotazione per il movimento dell'EE del robot**

Osservazione: il problema è che tu non hai una terna di riferimento a tutti gli effetti, alla fine ti basi sempre sulla terna precedente in quanto a posizione e ad orientamento pure